In [ ]:
import numpy as np

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
pathname = "cifar-10-batches-py"

In [4]:
def get_labelnames(dict):
    return [item.decode("utf8") for item in dict[b"label_names"]]

In [5]:
def mean_squared_error(y,t):
    return np.sum((y-t)**2)/2

In [6]:
def numerical_diff(f,x):
    h = 1e-4
    return (f(x+h)-f(x-h))/(2*h) 

In [7]:
def cross_entropy_error(y,t):
    delta = 1e-7 # log(0)を防ぐため
    if y.ndim == 1:
        y = y.reshape(1,y.size)
        t = t.reshape(1,t.size)
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y+delta))/batch_size

In [8]:
def numerical_gradient(f,x):
    h = 1e-4
    grad = np.zeros(x.shape)
    for i in range(len(x)):
        tmp = x[i]
        x[i] = x[i] + h
        fxhp = f(x)
        x[i] = tmp
        x[i] = x[i] - h
        fxhm = f(x)
        grad[i] = (fxhp - fxhm)/(2*h)
    return grad

In [9]:
labelname = get_labelnames(unpickle(pathname+"/batches.meta"))

In [10]:
data_arr = []
label_arr = []
DATA_BATCH_NUM = 5
for i in range(DATA_BATCH_NUM):
    data_batch = unpickle(pathname+"/data_batch_{}".format(i+1))
    data_arr.extend(data_batch[b'data'])
    label_arr += data_batch[b'labels']
data_arr = np.array(data_arr)
label_arr = np.array(label_arr)

In [11]:
def softmax(x):
    c = np.max(x)
    return np.exp(x-c)/np.sum(np.exp(x-c))

In [12]:
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.W) + self.b
        return out
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)
        return dx

In [13]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        return self.loss
    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

In [14]:
class Sigmoid:
    def __init__(self):
        self.out = None
    def forward(self,x):
        out = 1/(1+np.exp(-x))
        self.out = out
        return out
    def backward(self,dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [15]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [16]:
class TwoLayerNeuralNet():
    def __init__(self,input_size=3072,hidden_size=20,output_size=10,weight_init_std=0.01):
        self.params = {}
        self.params["W1"] = np.random.randn(input_size,hidden_size) * weight_init_std
        self.params["B1"] = np.zeros(hidden_size)
        self.params["W2"] = np.random.randn(hidden_size,output_size) * weight_init_std
        self.params["B2"] = np.zeros(output_size)
        self.layers = OrderedDict()
        self.layers["Affine1"] =  Affine(self.params['W1'], self.params['B1'])
        self.layers["Relu1"] = Relu()
        self.layers["Affine2"] =  Affine(self.params['W2'], self.params['B2'])
        self.lastLayer = SoftmaxWithLoss()
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        return self.lastLayer.forward(y,t)

    def gradient(self,x,t):
        self.loss(x,t)
        dout = self.lastLayer.backward(1) # 
        layers_tmp = list(self.layers.values())
        layers_tmp.reverse()
        for layer in layers_tmp:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'], grads['B1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['B2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        return grads
    
    

In [17]:
nn = TwoLayerNeuralNet(input_size=784, hidden_size=50, output_size=10)

In [18]:
def get_label_one_hot_arr(label_arr,label_num):
    t_batch = np.array([])
    batch_size = len(label_arr)
    for label in label_arr:
        t = np.zeros(label_num)
        t[label] = 1
        t_batch = np.append(t_batch,t)
    return t_batch.reshape(batch_size,label_num)

In [19]:
iters_num = 10000
train_size = data_arr.shape[0]
batch_size = 100
learning_rate = 0.1
loss_arr = []
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = nn.gradient(x_batch,t_batch)
    for key in ("W1","B1","W2","B2"):
        nn.params[key] -= learning_rate * grad[key]
    loss = nn.loss(x_batch,t_batch)
    loss_arr.append(loss)


NameError: name 'train_size' is not defined

In [24]:
accurate_num = 0
data_size = label_arr.shape[0]

In [101]:
t_train[1]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [105]:
nn.predict(x_train[0])

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [103]:
for i in range(60,100):
    
    # print([int(item) for item in softmax(nn.predict(x_train[i])) == np.max(softmax(nn.predict(x_train[i])))])
    print(t_train[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0. 0. 0. 0. 0. 1. 0. 0.

In [25]:
# for i in range(data_size):
#     if np.where(softmax(nn.predict(data_arr[i])) == np.max(softmax(nn.predict(data_arr[i])))) == np.where(softmax(nn.predict(data_arr[i])):
#         accurate_num += 1
# print(accurate_num/data_size)

0.09784


In [26]:
accurate_num

4892

In [117]:
data_size

50000